In [1]:
import numpy as np
import Robot
import matplotlib.pyplot as plt
import MyRobotMath as math
from MyRobotMath import SE3, quintic_time_scaling
from scipy.spatial.transform import Rotation as R

scara = Robot.SCARA(2,1,1)
se3 = SE3()
M = scara.zero
S = scara.S_tw
B = scara.B_tw
N = len(scara.joints)

In [2]:
desired = [0.6,0.4,1.2,0,0,-77] # 목표 자세의 (x,y,z,roll,pitch,yaw)
T_d = se3.pose_to_SE3(desired) # 목표 자세의 Tranformation Matrix
threshold = 1e-6 # 오차 범위
count = 0

init = [0,0,0,0]

while True:

    matexps_b = []
    matexps_s = []

    count += 1 # 연산 횟수 증가

    for i in range(N):
        matexps_b.append(se3.matexp(init[i],B[i],joint=scara.joints[i].type)) # Body Axis 기준 각 축의 Matrix Exponential
        matexps_s.append(se3.matexp(init[i],S[i],joint=scara.joints[i].type)) # Space Axis 기준 각 축의 Matrix Exponential

    T_sb = se3.matFK(M,matexps_b) # Forward Kinematics 적용 변환행렬
    estimated = []
    for i in range(3):
        estimated.append(T_sb[i,3].item()) # 현재 x, y, z
    
    eulerAngles = se3.CurrenntAngles(T_sb)
    for eulerAngle in eulerAngles:
        estimated.append(eulerAngle) # Euler 각도 추정값

    pos_err = np.array(desired[:3]) - np.array(estimated[:3]) # x, y, z 오차

    T_bd = np.dot(np.linalg.inv(T_sb),T_d) # Relative Trasformation Matrix
    J_b = se3.body_jacobian(M,matexps_b,matexps_s,S) # Body Jacobian
    J_pseudo = se3.j_inv(J_b) # Jacobian의 역행렬 (또는 의사역행렬)
    V_bd = se3.matlogm(T_bd) # Ralative Twist, 각도 오차

    theta = math.deg2rad(init,scara.joints)
    thetak = theta.reshape(N,1) + J_pseudo @ V_bd.reshape(6,1) # Newton Raphson Method
    thetak = math.rad2deg(thetak,scara.joints) # radian to degree
    init = math.theta_normalize(thetak,scara.joints) # 각도 정규화 (-180~180)

    if np.all(np.abs(pos_err) < threshold): # 오차가 임계값 이내면 break
        print(estimated)
        print(f"연산 횟수 : {count}, Joint Value : {init}")
        break


AttributeError: 'tuple' object has no attribute 'reshape'

In [5]:
end = init

matexps = []

for i in range(N):
    matexps.append(se3.matexp(scara.joints[i].type,end[i],B[i])) # Body Axis 기준 각 축의 Matrix Exponential

In [ ]:
for matexp in matexps:
    print(matexp)

In [ ]:
poses = [M]
T = M
for i in range(len(matexps)):
    T = T @ matexps[i]
    poses.append(T)

for pose in poses:
    print(pose)

In [6]:
def forward_kinematics_all_body(joint_values):
    T = scara.zero.copy()
    poses = [T.copy()]
    for joint, val, tw in zip(scara.joints, joint_values, scara.B_tw):
        T = T @ se3.matexp(joint.type, val, tw)
        poses.append(T.copy())
    return poses

In [7]:
poses = forward_kinematics_all_body(end)

In [ ]:
for i, pose in enumerate(poses):
    print(f"T{i}:\n", pose)

In [ ]:
theta_start = np.array([135,45,0,0.2])
theta_end = np.array(init)
print(theta_start,theta_end)

In [10]:
T = 4.0
N = 200

trajectory = []
velocity = []
accelation = []

for i in range(N):
    t = i / N * T
    s, s_dot, s_ddot = quintic_time_scaling(t, T)
    theta_desired = theta_start + s*(theta_end-theta_start)
    theta_dot = s_dot*(theta_end-theta_start)
    theta_ddot = s_ddot*(theta_end-theta_start)
    trajectory.append(theta_desired)
    velocity.append(theta_dot)
    accelation.append(theta_ddot)

trajectory = np.array(trajectory).T  # shape = (4, N)
velocity = np.array(velocity).T
accelation = np.array(accelation).T

In [ ]:
plt.figure(figsize=(10, 6))
for i, joint in enumerate(["θ1", "θ2", "θ3", "d4"]):
    plt.plot(np.linspace(0, T, N), trajectory[i], label=joint)

plt.xlabel("Time (s)")
plt.ylabel("Joint Value")
plt.title("SCARA Joint Trajectory")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
for i, joint in enumerate(["θ1", "θ2", "θ3", "d4"]):
    plt.plot(np.linspace(0, T, N), velocity[i], label=joint)

plt.xlabel("Time (s)")
plt.ylabel("Velocity Value")
plt.title("SCARA Joint Trajectory")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

In [15]:
print(np.zeros(3))

[0. 0. 0.]


In [6]:
desired = [10,-134,24,0,0,-135]
start = [150,0,50,0,0,0]

In [7]:
T1 = se3.pose_to_SE3(desired)
T0 = se3.pose_to_SE3(start)

In [ ]:
print(np.linalg.inv(T0)@T1)

In [4]:
theta = 2
print(np.rad2deg(theta))

114.59155902616465


In [8]:
arr = [0,0,0]

In [9]:
x,y,z=arr

In [12]:
test = list(input())

In [ ]:
desired = []
coodinate = ['x','y','z','roll','pitch','yaw']
for i, comp in enumerate(coodinate):
    desired.append(float(input(f"{comp} : ")))
    

In [15]:
print(desired)

[1.0, 2.0, 3.0, 4.0, 5.0, 6.0]


In [12]:
rotmat = R.from_euler('zyx',[45,45,45]).as_matrix()
print(rotmat)

[[ 0.27596319 -0.44699833  0.85090352]
 [ 0.82735079 -0.34012228 -0.44699833]
 [ 0.48921876  0.82735079  0.27596319]]


In [13]:
euler = R.from_matrix(rotmat).as_euler('zyx', degrees=True)
print(euler)

[58.31007809 58.31007809 58.31007809]


In [11]:
rotmat = R.from_euler('zyx',[-54.7355,-9.735565,-30]).as_matrix()
print(rotmat)

[[ 0.22850753  0.92426497  0.30580807]
 [ 0.07722575 -0.3303379   0.94069817]
 [ 0.97047437 -0.19134036 -0.14686175]]
